## MLlib PySpark Customer Churn

The data is saved as customer_churn.csv. Here are the fields and their definitions:

* Name : Name of the latest contact at Company
* Age: Customer Age
* Total_Purchase: Total Ads Purchased
* Account_Manager: Binary 0=No manager, 1= Account manager assigned
* Years: Totaly Years as a customer
* Num_sites: Number of websites that use the service.
* Onboard_date: Date that the name of the latest contact was onboarded
* Location: Client HQ Address
* Company: Name of Client Company

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('churn').getOrCreate()

In [0]:
# read pyspark dataset
df = spark.read.csv('dbfs:/FileStore/customer_churn.csv', inferSchema=True, header=True)

In [0]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [0]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [0]:
print((df.count(), len(df.columns)))

(900, 10)


In [0]:
from pyspark.sql.functions import col, sum as _sum

In [0]:
# check missing values
missing_val = df.select([_sum(col(c).isNull().cast('int')).alias(c) for c in df.columns])
missing_val.show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|    0|  0|             0|              0|    0|        0|           0|       0|      0|    0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [0]:
# check duplicated
duplicated = df.exceptAll(df.dropDuplicates())
duplicated.show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [0]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
df.select('Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites','Churn').describe().show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|              900|              900|               900|              900|               900|                900|
|   mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|             22.0|            100.0|                 0|              1.0|               3.0|                  0|
|    max|             65.0|         18026.01|                 1|             9.15|              14.0|                  1|
+-------+---------------

## Format for MLlib

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
assembler = VectorAssembler(inputCols= ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], 
                            outputCol='features')

In [0]:
output = assembler.transform(df)

In [0]:
final_df = output.select('features', 'Churn')

## Train Test Split

In [0]:
train, test = final_df.randomSplit([0.7, 0.3])

## Model Logistic Regression

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
log_reg = LogisticRegression(labelCol='Churn')

In [0]:
logModel = log_reg.fit(train)

In [0]:
predictions = logModel.transform(test)

In [0]:
predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.58239094354330...|[0.98987319463096...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.70817174117243...|[0.99105939973604...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.47131084586880...|[0.99581189846527...|       0.0|
|[28.0,11245.38,0....|    0|[3.68659084795130...|[0.97555523857261...|       0.0|
|[29.0,13255.05,1....|    0|[4.15900959955481...|[0.98461730095622...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.06453390440724...|[0.99768154682215...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.99553365872223...|[0.98193473229697...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.09433580127316...|[0.95665849682304...|       0.0|
|[30.0,13473.35,0....|    0|[2.74629985978277...|[0.93970403933739...|       0.0|
|[31.0,7073.61,0

## Evaluations

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [0]:
AUC = eval.evaluate(predictions)

In [0]:
AUC

0.8336329465598954

## Predict on brand new unlabeled data

In [0]:
model_full_data = log_reg.fit(final_df)

In [0]:
new_customers = spark.read.csv('dbfs:/FileStore/new_customers.csv', inferSchema=True, header=True)

In [0]:
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [0]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [0]:
assebler_new_customer = assembler.transform(new_customers)

In [0]:
assebler_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
predictions_new_data = model_full_data.transform(assebler_new_customer)

In [0]:
predictions_new_data.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!

## Good job..!!